In [106]:
import pandas as pd
import re
from datetime import datetime
import numpy as np

In [107]:
year = '2020'
data = pd.read_csv('data/raw-data/espn-raw-pbp-'+year+'.csv',)

In [108]:
data.columns

Index(['Time', 'EventDescription', 'Score', 'EventTeam', 'HomeOrAway',
       'GameID', 'Date', 'GameType', 'WinningTeam', 'AwayTeam', 'HomeTeam',
       'ShotOutcome', 'Quarter', 'pid', 'ShotID', 'xcoord', 'ycoord',
       'AwayScore', 'HomeScore'],
      dtype='object')

In [109]:
df = data[['Date','GameID','GameType','HomeTeam','AwayTeam','Quarter','Time','Score','HomeScore','AwayScore','WinningTeam','EventDescription','pid','ShotOutcome','ShotID','xcoord','ycoord']]

In [110]:
#add all of the columns so they are in the same order for all years
list_of_columns = ['Event', 'EventTeam', 'EventPlayer', 'Shooter',
       'SpecificShotType', 'ShotType', 'ShotOutcome', 'ShooterNumPTS',
       'Foul', 'FoulType', 'FoulNumFouls', 'FoulNumShots', 'TechnicalType',
       'Rebounder', 'ReboundType', 'RebounderNumOffRebounds', 'RebounderNumDefRebounds',
       'Assist', 'AssistNumAST', 'Block','BlockNumBLK', 
       'Turnover', 'TurnoverType', 'TurnoverNumTO', 'Steal', 'StealNumSTL',
       'SubstitutionOut', 'SubstitutionIn',
       'JumpBallPlayer1', 'JumpBallPlayer2', 'JumpBallPlayerPoss', 'JumpBallTeamPoss', 
       'Timeout', 'TimeoutType', 'StoppageType', 'InstantReplayType',
        'Violation','ViolationType']

for new_column in list_of_columns:
    df[new_column] = ""

<ipython-input-110-0297f4888416>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[new_column] = ""


In [111]:
df

,Date,GameID,GameType,HomeTeam,AwayTeam,Quarter,Time,Score,HomeScore,AwayScore,...,JumpBallPlayer1,JumpBallPlayer2,JumpBallPlayerPoss,JumpBallTeamPoss,Timeout,TimeoutType,StoppageType,InstantReplayType,Violation,ViolationType
0,2020-07-25T16:00Z,401230733,regular season,NY,SEA,1,10:00,0 - 0,0,0,...,,,,,,,,,,
1,2020-07-25T16:00Z,401230733,regular season,NY,SEA,1,9:48,0 - 0,0,0,...,,,,,,,,,,
2,2020-07-25T16:00Z,401230733,regular season,NY,SEA,1,9:43,0 - 0,0,0,...,,,,,,,,,,
3,2020-07-25T16:00Z,401230733,regular season,NY,SEA,1,9:39,0 - 0,0,0,...,,,,,,,,,,
4,2020-07-25T16:00Z,401230733,regular season,NY,SEA,1,9:33,2 - 0,0,2,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48862,2020-09-12T19:00Z,401230819,regular season,LA,LV,4,21.2,84 - 70,70,84,...,,,,,,,,,,
48863,2020-09-12T19:00Z,401230819,regular season,LA,LV,4,2.3,84 - 70,70,84,...,,,,,,,,,,
48864,2020-09-12T19:00Z,401230819,regular season,LA,LV,4,0.6,84 - 70,70,84,...,,,,,,,,,,
48865,2020-09-12T19:00Z,401230819,regular season,LA,LV,4,0.0,84 - 70,70,84,...,,,,,,,,,,


In [112]:
for row in range(len(df)):
    print(df.loc[row,'EventDescription'])

Breanna Stewart vs. Kiah Stokes (Jewell Loyd gains possession)
Breanna Stewart misses 19-foot jumper
Amanda Zahui B. defensive rebound
Amanda Zahui B. bad pass (Alysha Clark steals)
Natasha Howard makes layup (Alysha Clark assists)
Layshia Clarendon misses 27-foot three point jumper
Natasha Howard defensive rebound
Alysha Clark misses 17-foot jumper
Sabrina Ionescu defensive rebound
Kiah Stokes makes 5-foot two point shot (Amanda Zahui B. assists)
Jewell Loyd misses 25-foot three point jumper
Amanda Zahui B. defensive rebound
Layshia Clarendon bad pass (Natasha Howard steals)
Breanna Stewart makes layup (Sue Bird assists)
Amanda Zahui B. makes jumper (Layshia Clarendon assists)
Sue Bird misses 26-foot three point jumper
Sabrina Ionescu defensive rebound
Layshia Clarendon makes layup (Amanda Zahui B. assists)
Natasha Howard makes two point shot
Amanda Zahui B. misses 28-foot three point jumper
Alysha Clark defensive rebound
Sue Bird misses 17-foot pullup jump shot
Kia Nurse defensive re

A'ja Wilson defensive rebound
Jackie Young makes layup (Lindsay Allen assists)
Cheyenne Parker lost ball turnover (A'ja Wilson steals)
Dearica Hamby misses 23-foot three point shot
Stefanie Dolson defensive rebound
Kahleah Copper makes jumper
A'ja Wilson makes layup (Lindsay Allen assists)
Chicago 20 Sec. timeout
Carolyn Swords enters the game for A'ja Wilson
Courtney Vandersloot enters the game for Gabby Williams
Ruthy Hebard enters the game for Stefanie Dolson
Azura Stevens enters the game for Cheyenne Parker
Ruthy Hebard offensive foul
Ruthy Hebard turnover
Carolyn Swords misses 10-foot jumper
Courtney Vandersloot defensive rebound
Allie Quigley makes 17-foot jumper
Carolyn Swords offensive charge
Carolyn Swords turnover
A'ja Wilson enters the game for Carolyn Swords
Courtney Vandersloot bad pass (Dearica Hamby steals)
A'ja Wilson makes 6-foot layup (Angel McCoughtry assists)
Azura Stevens misses 20-foot jump bank shot
Jackie Young defensive rebound
Jackie Young bad pass (Courtney V

Jordin Canada misses layup
Sylvia Fowles defensive rebound
Napheesa Collier bad pass (Breanna Stewart steals)
Breanna Stewart makes layup (Epiphanny Prince assists)
Crystal Dangerfield out of bounds bad pass turnover
Epiphanny Prince makes 16-foot jumper
Napheesa Collier lost ball turnover (Epiphanny Prince steals)
Epiphanny Prince lost ball turnover (Lexie Brown steals)
Napheesa Collier misses 24-foot three point jumper
Jordin Canada defensive rebound
Seattle Full timeout
Breanna Stewart misses 24-foot three point jumper
Sylvia Fowles defensive rebound
End of the 2nd Quarter
Sue Bird enters the game for Epiphanny Prince
Natasha Howard enters the game for Jordin Canada
Jewell Loyd enters the game for Ezi Magbegor
Shenise Johnson enters the game for Crystal Dangerfield
Damiris Dantas enters the game for Bridget Carleton
Lexie Brown personal foul
Jordin Canada enters the game for Sue Bird
Alysha Clark misses 25-foot three point jumper
Jordin Canada offensive rebound
Jewell Loyd misses 23

Epiphanny Prince personal foul
Myisha Hines-Allen enters the game for Emma Meesseman
Sami Whitcomb enters the game for Breanna Stewart
Crystal Langhorne enters the game for Alysha Clark
Ariel Atkins misses 25-foot three point jumper
Epiphanny Prince defensive rebound
Mercedes Russell makes layup (Sami Whitcomb assists)
Ariel Atkins makes 26-foot three point jumper (Myisha Hines-Allen assists)
Mercedes Russell offensive charge
Mercedes Russell turnover
Shey Peddy misses 26-foot three point jumper
Mercedes Russell defensive rebound
Epiphanny Prince misses 19-foot jumper
Alaina Coates defensive rebound
Myisha Hines-Allen misses 25-foot three point jumper
Washington offensive team rebound
End of the 1st Quarter
Emma Meesseman enters the game for Ariel Atkins
Kiara Leslie enters the game for Alaina Coates
Shey Peddy makes 26-foot three point jumper (Myisha Hines-Allen assists)
Myisha Hines-Allen personal foul
Epiphanny Prince misses 24-foot three point jumper
Myisha Hines-Allen defensive re

Jazmine Jones bad pass (Betnijah Laney steals)
Joyner Holmes personal foul
Official timeout
Layshia Clarendon enters the game for Leaonna Odom
Betnijah Laney offensive charge
Betnijah Laney turnover
Jocelyn Willoughby makes 25-foot three point jumper (Layshia Clarendon assists)
Jazmine Jones shooting foul
Elizabeth Williams makes free throw 1 of 2
Leaonna Odom enters the game for Jazmine Jones
Elizabeth Williams makes free throw 2 of 2
Layshia Clarendon makes 24-foot three point jumper (Kylee Shook assists)
Layshia Clarendon shooting foul
Chennedy Carter makes free throw 1 of 2
Chennedy Carter makes free throw 2 of 2
Elizabeth Williams shooting foul
Kylee Shook makes free throw 1 of 2
Kiah Stokes enters the game for Joyner Holmes
Kylee Shook misses free throw 2 of 2
Jocelyn Willoughby offensive rebound
Layshia Clarendon misses 6-foot jumper
Leaonna Odom offensive rebound
Jocelyn Willoughby makes 27-foot three point jumper (Leaonna Odom assists)
Chennedy Carter makes driving layup (Eliz

...

Marina Mabrey personal foul
Washington 20 Sec. timeout
Satou Sabally personal foul
Emma Meesseman misses free throw 1 of 2
Washington offensive team rebound
Emma Meesseman makes free throw 2 of 2
Arike Ogunbowale makes 25-foot three point jumper
Washington 20 Sec. timeout
Bella Alarie enters the game for Marina Mabrey
Emma Meesseman misses 19-foot jumper
Washington offensive team rebound
End of the 4th Quarter
Marina Mabrey enters the game for Bella Alarie
Satou Sabally vs. Myisha Hines-Allen
Ariel Atkins makes layup (Myisha Hines-Allen assists)
Marina Mabrey out of bounds bad pass turnover
Marina Mabrey shooting foul
Leilani Mitchell makes free throw 1 of 2
Leilani Mitchell makes free throw 2 of 2
Arike Ogunbowale makes 26-foot three point jumper
Emma Meesseman misses 19-foot jumper
Satou Sabally defensive rebound
Arike Ogunbowale makes 17-foot jumper
Satou Sabally personal foul
Emma Meesseman misses 8-foot hook shot
Kayla Thornton defensive rebound
Marina Mabrey makes layup
Myisha Hi

A'ja Wilson misses 8-foot two point shot
Angel McCoughtry offensive rebound
Teaira McCowan shooting foul
Angel McCoughtry makes free throw 1 of 2
Angel McCoughtry makes free throw 2 of 2
Julie Allemand bad pass (Angel McCoughtry steals)
Angel McCoughtry misses driving layup
Candice Dupree defensive rebound
Kennedy Burke makes driving layup (Julie Allemand assists)
Teaira McCowan blocks A'ja Wilson's layup
A'ja Wilson offensive rebound
A'ja Wilson misses layup
A'ja Wilson offensive rebound
A'ja Wilson misses two point shot
Teaira McCowan defensive rebound
Kelsey Mitchell makes 9-foot jumper (Teaira McCowan assists)
Angel McCoughtry misses 19-foot pullup jump shot
Teaira McCowan defensive rebound
Kelsey Mitchell misses driving layup
Candice Dupree offensive rebound
Kennedy Burke misses 25-foot three point jumper
Kayla McBride defensive rebound
Candice Dupree personal foul
Kayla McBride misses 18-foot jumper
Julie Allemand defensive rebound
Kelsey Mitchell bad pass (A'ja Wilson steals)
Li

Satou Sabally out of bounds bad pass turnover
Sami Whitcomb misses 23-foot three point jumper
Seattle offensive team rebound
Sue Bird enters the game for Jewell Loyd
Satou Sabally shooting foul
Breanna Stewart makes free throw 1 of 2
Breanna Stewart makes free throw 2 of 2
Tyasha Harris misses 23-foot three point jumper
Mercedes Russell defensive rebound
Jordin Canada misses 16-foot jumper
Satou Sabally defensive rebound
Satou Sabally misses three point jumper
Jordin Canada defensive rebound
Sami Whitcomb makes 23-foot three point jumper (Jordin Canada assists)
Katie Lou Samuelson misses 24-foot three point jumper
Sami Whitcomb defensive rebound
Breanna Stewart makes 4-foot layup (Sue Bird assists)
Satou Sabally misses 11-foot two point shot
Dallas offensive team rebound
End of the 3rd Quarter
Arike Ogunbowale enters the game for Bella Alarie
Sue Bird misses 24-foot three point pullup jump shot
Seattle offensive team rebound
Jordin Canada makes 3-foot two point shot (Breanna Stewart as

Elizabeth Williams bad pass (Kaila Charles steals)
Kaila Charles makes layup
End of the 4th Quarter
End of Game
Cheyenne Parker vs. Bella Alarie (Tyasha Harris gains possession)
Marina Mabrey lost ball turnover (Courtney Vandersloot steals)
Ruthy Hebard makes layup (Courtney Vandersloot assists)
Tyasha Harris misses three point jumper
Katie Lou Samuelson offensive rebound
Marina Mabrey misses 11-foot jumper
Bella Alarie offensive rebound
Marina Mabrey makes 11-foot jumper
Ruthy Hebard makes 3-foot layup (Kahleah Copper assists)
Tyasha Harris misses driving layup
Ruthy Hebard defensive rebound
Courtney Vandersloot makes driving layup
Arike Ogunbowale makes 16-foot pullup jump shot
Ruthy Hebard makes hook shot (Kahleah Copper assists)
Tyasha Harris makes driving layup
Kahleah Copper shooting foul
Tyasha Harris misses free throw 1 of 1
Ruthy Hebard defensive rebound
Kahleah Copper misses driving layup
Arike Ogunbowale defensive rebound
Katie Lou Samuelson misses 25-foot three point jumper

Riquna Williams makes free throw 2 of 2
Nneka Ogwumike blocks Jackie Young 's 8-foot jumper
Las Vegas offensive team rebound
Dearica Hamby misses 20-foot jumper
Candace Parker defensive rebound
Sugar Rodgers shooting foul
Candace Parker makes free throw 1 of 2
Candace Parker makes free throw 2 of 2
Carolyn Swords vs. Candace Parker (Riquna Williams gains possession)
Carolyn Swords lost ball turnover (Candace Parker steals)
Brittney Sykes makes layup (Riquna Williams assists)
Sugar Rodgers lost ball turnover (Riquna Williams steals)
Sugar Rodgers personal foul
Riquna Williams makes free throw 1 of 2
Kayla McBride enters the game for Sugar Rodgers
Riquna Williams makes free throw 2 of 2
Jackie Young makes driving layup
Candace Parker makes 18-foot pullup jump shot (Riquna Williams assists)
End of the 3rd Quarter
Angel McCoughtry enters the game for Danielle Robinson
A'ja Wilson enters the game for Carolyn Swords
A'ja Wilson blocks Brittney Sykes 's 4-foot layup
Las Vegas defensive team r

In [113]:
shot = ['misses','makes']
rebound = ['offensive rebound','defensive rebound','offensive team rebound','defensive team rebound']
foul = ['personal foul','shooting foul','loose ball foul','offensive charge',
        'flagrant foul type 1','double personal foul','offensive foul',
        'away from play foul','personal take foul','clear path foul']
tech_foul = ['technical foul','delay techfoul','Double technical foul','defensive 3-seconds']
turnover = ['turnover','lost ball turnover','bad pass','traveling','out of bounds bad pass turnover',
              'double dribble turnover','palming turnover','steps out of bounds','out of bounds lost ball',
              'shot clock turnover']
substitution = ['enters the game for']
timeout = ['Full timeout','Official timeout','20 Sec. timeout']
block = ['blocks']
violation = ['5 second violation','delay of game violation','kicked ball violation',
              'jump ball violation','lane violation','double lane violation','violation']
jump_ball = ['vs.']
ejection = ['ejected']


event2_list = ['assists','steals','gains possession']

game_events = ['End of the 1st Quarter','End of the 2nd Quarter','End of the 3rd Quarter','End of the 4th Quarter',
               'End of the 1st Overtime','End of the 2nd Overtime','End of the 3rd Overtime',
               'End of the 1st  Overtime','End of the 2nd  Overtime',
               'End of Game']

shot_regex = '|'.join(shot)
rebound_regex = '|'.join(rebound)
#foul = '|'.join(foul)
#turnover = '|'.join(turnover)
#substitution = '|'.join(substitution)
#timeout = '|'.join(timeout)
#block = '|'.join(block)
#violation = '|'.join(violation)
#jump_ball = '|'.join(jump_ball)
#
#event2_list = '|'.join(event2_list)
#game_events = '|'.join(game_events)

#instead make them all into one list to match to so then I can use the in operator to check if it's in one of these groups
all_events = [shot,rebound,foul,tech_foul,turnover,substitution,timeout,block,violation,jump_ball,ejection,event2_list,game_events]
all_events = [item for sublist in all_events for item in sublist]
all_events

['misses',
 'makes',
 'offensive rebound',
 'defensive rebound',
 'offensive team rebound',
 'defensive team rebound',
 'personal foul',
 'shooting foul',
 'loose ball foul',
 'offensive charge',
 'flagrant foul type 1',
 'double personal foul',
 'offensive foul',
 'away from play foul',
 'personal take foul',
 'clear path foul',
 'technical foul',
 'delay techfoul',
 'Double technical foul',
 'defensive 3-seconds',
 'turnover',
 'lost ball turnover',
 'bad pass',
 'traveling',
 'out of bounds bad pass turnover',
 'double dribble turnover',
 'palming turnover',
 'steps out of bounds',
 'out of bounds lost ball',
 'shot clock turnover',
 'enters the game for',
 'Full timeout',
 'Official timeout',
 '20 Sec. timeout',
 'blocks',
 '5 second violation',
 'delay of game violation',
 'kicked ball violation',
 'jump ball violation',
 'lane violation',
 'double lane violation',
 'violation',
 'vs.',
 'ejected',
 'assists',
 'steals',
 'gains possession',
 'End of the 1st Quarter',
 'End of the

In [114]:
bad_rows = list()
temp_df_dts = []
for row in range(len(df['EventDescription'])):
    string = df.loc[row,'EventDescription']
    if pd.isnull(string):
        continue
    tracker = 0
    for event_type in all_events:
        if event_type in string:
            df.loc[row,'SpecificEvent'] = event_type
            #print("Match!!:"+event_type)
            tracker += 1 #track whether we have matched to an event in our list
            
            if event_type in shot:
                df.loc[row,'Event'] = 'Shot'
                df.loc[row,'ShotOutcome'] = event_type
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<shot_type>[\w\s\-]*)?([:\s]+)?(?P<paren>\([\w\s\-\.']*\))?", string)
                df.loc[row,'EventPlayer'] = match.group('player').strip()
                df.loc[row,'Shooter'] = match.group('player').strip()
                df.loc[row,'SpecificEvent'] = match.group('shot_type').strip()
                if 'three' in match.group('shot_type'):
                    df.loc[row,'ShotType'] = '3pt Shot'
                elif 'free' in match.group('shot_type'):
                    df.loc[row,'ShotType'] = 'Free Throw'
                else:
                    df.loc[row,'ShotType'] = '2pt Shot'
                
                
            if event_type in rebound:
                df.loc[row,'Event'] = 'Rebound'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<shot_type>[\w\s\-]*)?([:\s]+)?(?P<paren>\([\w\s\-\.']*\))?", string)
                if 'team' in event_type:
                    df.loc[row,'EventPlayer'] = "Team"
                else:
                    df.loc[row,'EventPlayer'] = match.group('player').strip()
                    df.loc[row,'Rebounder'] = match.group('player').strip()
                if 'offensive' in event_type:
                    df.loc[row,'ReboundType'] = 'Offensive'
                if 'defensive' in event_type:
                    df.loc[row,'ReboundType'] = 'Defensive'
                
            if event_type in foul:
                df.loc[row,'Event'] = 'Foul'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")", string)
                df.loc[row,'FoulType'] = event_type
                df.loc[row,'EventPlayer'] = match.group('player').strip()
                                 
            if event_type in tech_foul:            
                df.loc[row,'Event'] = 'Technical Foul'
                df.loc[row,'TechnicalType'] = event_type
                if "Double" in string:
                    match = re.match(r"Double technical foul: (?P<player>[\w\s\-\.']*)([:\s]+)? and (?P<player2>[\w\s\-\.']*)([:\s]+)?", string)
                    df.loc[row,'EventPlayer'] = match.group('player2').strip()
                    dtrow = df.iloc[row]
                    temp_df_dts.append(list(dtrow))
                    df.loc[row,'EventPlayer'] = match.group('player').strip()
                else:
                    string2 = re.sub("([\(\[]).*?([\)\]])", "", string)
                    if event_type in string2:
                        match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+").*", string2)
                        df.loc[row,'EventPlayer'] = match.group('player').strip()
                                 
            if event_type in turnover:
                df.loc[row,'Event'] = 'Turnover'
                df.loc[row,'TurnoverType'] = event_type
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")", string)
                df.loc[row,'Turnover'] = match.group('player').strip()
                df.loc[row,'EventPlayer'] = match.group('player').strip()
                                 
            if event_type in substitution:
                df.loc[row,'Event'] = 'Substitution'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<player2>[\w\s\-\.']*)([:\s]+)?", string)
                df.loc[row,'SubstitutionOut'] = match.group('player2').strip()
                df.loc[row,'SubstitutionIn'] = match.group('player').strip()
                                 
            if event_type in timeout:
                df.loc[row,'Event'] = 'Timeout'
                df.loc[row,'TimeoutType'] = event_type
                match = re.match(r"(?P<team>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")", string)
                df.loc[row,'EventTeam'] = match.group('team').strip()
                
            if event_type in block:
                df.loc[row,'Event'] = 'Block'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<player2>[\w\s\-\.']*)?([:\s]+)?\'s([:\s]+)?(?P<shot_type>[\w\s\-]*)?", string)
                df.loc[row,'EventPlayer'] = match.group('player').strip()
                df.loc[row,'Shooter'] = match.group('player2').strip()
                df.loc[row,'SpecificShotType'] = match.group('shot_type').strip()
                if 'three' in match.group('shot_type'):
                    df.loc[row,'ShotType'] = '3pt Shot'
                else:
                    df.loc[row,'ShotType'] = '2pt Shot'
                df.loc[row,'ShotOutcome'] = "blocked"
                
            if event_type in violation:
                df.loc[row,'Event'] = 'Violation'
                df.loc[row,'ViolationType'] = event_type
                
            if event_type in jump_ball:
                df.loc[row,'Event'] = 'Jump Ball'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")([:\s]+)?(?P<player2>[\w\s\-\.']*)?([:\s]+)?.*", string)
                df.loc[row,'JumpBallPlayer1'] = match.group('player').strip()
                df.loc[row,'JumpBallPlayer2'] = match.group('player2').strip()
                #player possession is down in secondary events
             
            if event_type in ejection:
                df.loc[row,'Event'] = 'Ejection'
                match = re.match(r"(?P<player>[\w\s\-\.']*)([:\s]+)?(?P<event>"+event_type+")", string)
                df.loc[row,'Player'] = match.group('player').strip()
                
            if event_type in event2_list:
                match = re.match(r".*\((?P<paren>[\w\s\-\.']*)"+event_type+"\)", string)
                if event_type == "assists":
                    df.loc[row,'Assist'] = match.group('paren').strip()
                if event_type == "steals":
                    df.loc[row,'Steal'] = match.group('paren').strip()
                if event_type == "gains possession":
                    df.loc[row,'JumpBallPlayerPoss'] = match.group('paren').strip()
                
            if event_type in game_events:
                df.loc[row,'Event'] = 'End of Period'
            
    if tracker == 0:
        bad_rows.append(row)
        print('No match!  ' + string)

No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  (03:27) [New York]
No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  foul
No match!  foul


In [115]:
string

'End of Game'

In [ ]:
#need to handle double techs

In [116]:
for row in range(len(df['EventDescription'])):
    string = df.loc[row,'EventDescription']
    if pd.isnull(string):
        continue
    tracker = 0
    for event_type in all_events:
        pass
                
                    



In [88]:
string2

'Satou Sabally defensive 3-seconds ()'

In [ ]:
temp_df_dts = pd.DataFrame(temp_df_dts, columns=df.columns)
df = df.append(temp_df_dts, ignore_index=False)

In [73]:
df.loc[bad_rows,'EventDescription']
df.loc[bad_rows,]

,Date,GameID,GameType,HomeTeam,AwayTeam,Quarter,Time,Score,HomeScore,AwayScore,...,JumpBallPlayerPoss,JumpBallTeamPoss,Timeout,TimeoutType,StoppageType,InstantReplayType,Violation,ViolationType,SpecificEvent,Player
1133,2020-07-25T21:00Z,401230735,regular season,WSH,IND,4,4:37,62 - 95,95,62,...,,,,,,,,,NaN,NaN
2783,2020-07-28T23:00Z,401230464,regular season,CONN,WSH,3,37.8,74 - 71,71,74,...,,,,,,,,,NaN,NaN
3342,2020-07-29T02:00Z,401230466,regular season,SEA,MIN,1,1:20,11 - 16,16,11,...,,,,,,,,,NaN,NaN
8472,2020-08-02T00:00Z,401230758,regular season,SEA,LA,4,5:56,63 - 68,68,63,...,,,,,,,,,NaN,NaN
11747,2020-08-06T02:00Z,401230767,regular season,LA,IND,1,3:27,11 - 17,17,11,...,,,,,,,,,NaN,NaN
13965,2020-08-07T23:00Z,401230772,regular season,WSH,NY,4,4:52,65 - 56,56,65,...,,,,,,,,,NaN,NaN
23326,2020-08-15T22:00Z,401230796,regular season,MIN,NY,2,3:27,27 - 34,34,27,...,,,,,,,,,NaN,NaN
26580,2020-08-20T01:00Z,401230826,regular season,MIN,DAL,4,33.1,79 - 89,89,79,...,,,,,,,,,NaN,NaN
27433,2020-08-21T00:00Z,401230828,regular season,IND,SEA,1,3:48,13 - 12,12,13,...,,,,,,,,,NaN,NaN
28675,2020-08-22T00:00Z,401230830,regular season,DAL,WSH,1,52.3,15 - 25,25,15,...,,,,,,,,,NaN,NaN


In [ ]:
g = df.groupby(['pid','EventPlayer'])
d = g.last()
d = d.reset_index()
players = {}
for row in range(len(d)):
    players[d.loc[row,'pid']] = d.loc[row,'EventPlayer']
    if d.loc[row,'EventPlayer'] == 'Team':
        players[d.loc[row,'pid']] = d.loc[row,'EventTeam']

In [74]:
#df.to_csv('test.csv')
df.to_csv('data/wnba-pbp-'+year+'.csv')